In [1]:
## import

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.feature_selection import RFECV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR
from bayes_opt import BayesianOptimization as BO
from IPython.display import display
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
import xgboost as XGB
from sklearn.ensemble import GradientBoostingRegressor as GBR
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

%matplotlib inline
plt.style.use('ggplot')

In [149]:
df = pd.read_csv('train_new_features.csv', index_col='id')
df.head()

,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_size_calc,avg_EA,avg_EN,avg_HOMO,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,...,0.201799,-0.020559,0.172588,-0.277587,-0.593489,0.028551,4.405298,0.282055,0.147109,0.110364
2,194,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,...,0.201425,-0.020521,0.172268,-0.277073,-0.592391,0.028498,4.397146,0.281534,0.146837,0.110160
3,227,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,...,0.184712,-0.023364,0.168444,-0.276418,-0.591693,0.033063,3.578027,0.299762,0.150104,0.112850
4,167,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,...,0.223062,-0.027645,0.168980,-0.278012,-0.584846,0.045990,5.004543,0.288110,0.150574,0.114528
5,194,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,...,0.253691,-0.015645,0.152321,-0.233002,-0.483778,0.028951,7.336285,0.175926,0.114532,0.087010


In [150]:
def rmsle(actual, predicted):
    return np.square(np.log(predicted + 1) - np.log(actual + 1)).mean() ** 0.5
my_score = make_scorer(rmsle, greater_is_better=False)

In [151]:
def select_model(data, model):
    
    exclude_feature = ['sg', 'Eg', 'Ef', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al',]
    features = list(data.drop(exclude_feature, axis=1))
    X = data.drop(exclude_feature, axis=1).values

    y_g = data['Eg'].values
    y_f = data['Ef'].values
    
#    print(X.shape, y_g.shape, y_f.shape)
    params = {}
    models = {}
    
    search_params_RFR = {
     "n_estimators": Integer(50, 2000),
     'max_depth': Integer(2, 40),
     'min_samples_split': Integer(2, 15),
#     'max_features': Integer(2, 20)
     }
    
    search_params_GBR = {
     'learning_rate': Real(0.01, 0.5),
     'n_estimators': Integer(1000, 4000),
     'max_depth': Integer(2, 40),
     'min_samples_split': Integer(2, 15),
     'min_samples_leaf': Integer(2, 50),
     'min_weight_fraction_leaf': Real(0., .5),
     'max_features': Integer(2, 15)
     }
    
    params['RFR'] = search_params_RFR
    params['GBR'] = search_params_GBR
    
    models['RFR'] = RFR(max_features="sqrt")
    models['GBR'] = GBR()

    opt_g = BayesSearchCV(models[model], 
                      params[model], 
                      scoring=my_score,
                      cv=5,
                      n_iter=50, 
                      n_jobs=-1, 
                      verbose=10)
    opt_g.fit(X, y_g)
    
    opt_f = BayesSearchCV(models[model], 
                      params[model], 
                      scoring=my_score,
                      cv=5,
                      n_iter=50, 
                      n_jobs=-1, 
                      verbose=10)
    opt_f.fit(X, y_f)
    
    output = {}
    res_g = pd.DataFrame(opt_g.cv_results_).sort_values(['rank_test_score', 'param_max_depth'])
    res_f = pd.DataFrame(opt_f.cv_results_).sort_values(['rank_test_score', 'param_max_depth'])
    
    output["results_g"] = res_g
    output["results_f"] = res_f
    
    output["best_params_g"] = opt_g.best_params_
    output["best_params_f"] = opt_f.best_params_
    
    output["best_g"] = opt_g.best_estimator_
    output["best_f"] = opt_f.best_estimator_
    
    output["rmsle_g"] = abs(res_g['mean_test_score'].max())
    output["std_g"] = res_g.iloc[0]['std_test_score']
    
    output["rmsle_f"] = abs(res_f['mean_test_score'].max())
    output["std_f"] = res_f.iloc[0]['std_test_score']
    
    output["avg_rmsle"] = (output["rmsle_g"] + output["rmsle_f"])/2
    
    return output

In [153]:
sg = np.sort(df['sg'].unique())
train = {}
out_rfr = {}

for s in sg:
    train[s] = df.groupby('sg').get_group(s)
    out_rfr[s] = select_model(train[s], 'RFR')
#    avg_rmsle_rfr = np.append(avg_rmsle, out_rfr[s]["avg_rmsle"]) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] max_depth=9, min_samples_split=8, n_estimators=161 ..............
[CV] max_depth=9, min_samples_split=8, n_estimators=161 ..............
[CV] max_depth=9, min_samples_split=8, n_estimators=161 ..............
[CV] max_depth=9, min_samples_split=8, n_estimators=161 ..............
[CV]  max_depth=9, min_samples_split=8, n_estimators=161, score=-0.062460348772603, total=   0.4s
[CV]  max_depth=9, min_samples_split=8, n_estimators=161, score=-0.05184582578524069, total=   0.4s
[CV] max_depth=9, min_samples_split=8, n_estimators=161 ..............
[CV] max_depth=2, min_samples_split=14, n_estimators=150 .............
[CV]  max_depth=9, min_samples_split=8, n_estimators=161, score=-0.050119144788092825, total=   0.5s
[CV] max_depth=2, min_samples_split=14, n_estimators=150 .............
[CV]  max_depth=9, min_samples_split=8, n_estimators=161, score=-0.06386880306526602, total=   0.4s
[CV] max_depth=2, min_samples_split=14, n_es

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.9s


[CV]  max_depth=26, min_samples_split=9, n_estimators=77, score=-0.05142097150416025, total=   0.2s
[CV] max_depth=26, min_samples_split=9, n_estimators=77 ..............
[CV]  max_depth=26, min_samples_split=9, n_estimators=77, score=-0.06278969674738888, total=   0.2s
[CV] max_depth=26, min_samples_split=9, n_estimators=77 ..............
[CV]  max_depth=2, min_samples_split=14, n_estimators=150, score=-0.07587718155018555, total=   0.5s
[CV] max_depth=26, min_samples_split=9, n_estimators=77 ..............
[CV]  max_depth=2, min_samples_split=14, n_estimators=150, score=-0.06717177493539085, total=   0.5s
[CV] max_depth=24, min_samples_split=3, n_estimators=637 .............


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.3s


[CV]  max_depth=26, min_samples_split=9, n_estimators=77, score=-0.04960462100800899, total=   0.3s
[CV] max_depth=24, min_samples_split=3, n_estimators=637 .............
[CV]  max_depth=26, min_samples_split=9, n_estimators=77, score=-0.06482960615963897, total=   0.3s
[CV] max_depth=24, min_samples_split=3, n_estimators=637 .............
[CV]  max_depth=26, min_samples_split=9, n_estimators=77, score=-0.046604734499924624, total=   0.3s
[CV] max_depth=24, min_samples_split=3, n_estimators=637 .............
[CV]  max_depth=24, min_samples_split=3, n_estimators=637, score=-0.05202213410209999, total=   1.9s
[CV] max_depth=24, min_samples_split=3, n_estimators=637 .............
[CV]  max_depth=24, min_samples_split=3, n_estimators=637, score=-0.06313022805650013, total=   1.8s
[CV]  max_depth=24, min_samples_split=3, n_estimators=637, score=-0.05079151141171862, total=   1.8s
[CV]  max_depth=24, min_samples_split=3, n_estimators=637, score=-0.0657289442086385, total=   1.8s


...

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   13.4s finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] max_depth=2, min_samples_split=15, n_estimators=1917 ............
[CV] max_depth=2, min_samples_split=15, n_estimators=1917 ............
[CV] max_depth=2, min_samples_split=15, n_estimators=1917 ............
[CV] max_depth=2, min_samples_split=15, n_estimators=1917 ............
[CV]  max_depth=2, min_samples_split=15, n_estimators=1917, score=-0.05728731786308771, total=   5.0s
[CV] max_depth=2, min_samples_split=15, n_estimators=1917 ............
[CV]  max_depth=2, min_samples_split=15, n_estimators=1917, score=-0.04719406595655421, total=   5.1s
[CV] max_depth=2, min_samples_split=2, n_estimators=1905 .............
[CV]  max_depth=2, min_samples_split=15, n_estimators=1917, score=-0.05875318828457637, total=   5.1s
[CV] max_depth=2, min_samples_split=2, n_estimators=1905 .............
[CV]  max_depth=2, min_samples_split=15, n_estimators=1917, score=-0.052077665679161216, total=   5.2s
[CV] max_depth=2, min_samples_spli

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    9.4s remaining:    9.4s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    9.5s remaining:    4.1s


[CV]  max_depth=2, min_samples_split=2, n_estimators=1905, score=-0.058625505325497006, total=   2.6s
[CV]  max_depth=2, min_samples_split=2, n_estimators=1905, score=-0.05160717504735746, total=   2.6s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   12.0s finished


In [158]:
avg_rmsle_rfr = []
for s in sg:
    avg_rmsle_rfr = np.append(avg_rmsle_rfr, out_rfr[s]["avg_rmsle"]) 

print(avg_rmsle_rfr, avg_rmsle_rfr.mean())

[ 0.04566391  0.02614027  0.02285029  0.08012551  0.02655307  0.11238559] 0.0522864405396


In [188]:
for s in sg:
    res_f = out_rfr[s]["results_f"].sort_values(['rank_test_score', 'param_n_estimators'], ascending=[True,False])
    rank_f = res_f.groupby('rank_test_score').get_group(1)
#    display(rank_f["params"])
    for param in rank_f["params"]:
        print(param)
        
    res_g = out_rfr[s]["results_g"].sort_values(['rank_test_score', 'param_n_estimators'], ascending=[True,False])
#    display(res_g.head())
    params_f = res_f["params"].iloc[0]
    params_g = res_g["params"].iloc[0]
    best_f = RFR(**params_f)
    best_g = RFR(**params_g)
    
#    print(s, out_rfr[s]["best_params_g"], out_rfr[s]["best_params_f"])

{'max_depth': 29, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 29, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 30, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 40, 'min_samples_split': 14, 'n_estimators': 1992}
{'max_depth': 40, 'min_samples_split': 3, 'n_estimators': 1972}
{'max_depth': 34, 'min_samples_split': 9, 'n_estimators': 484}
{'max_depth': 21, 'min_samples_split': 9, 'n_estimators': 460}
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 117}
{'max_depth': 26, 'min_samples_split': 15, 'n_estimators': 50}
{'max_depth': 27, 'min_samples_split': 15, 'n_estimators': 50}
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 50}
{'max_depth': 28, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 40, 'min_samples_split': 15, 'n_estimators': 2000}
{'max_depth': 24, 'min_samples_split'

In [210]:
data_test = pd.read_csv('test_new_features.csv', index_col='id')

def predict(data):    
    best_model_g = {}
    best_model_f = {}    
    train_ex_feature = ['sg', 'Eg', 'Ef', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al',]
    test_ex_feature = ['sg', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al']
    df_test = {}
    
    for s in sg:
        X = train[s].drop(train_ex_feature, axis=1).values
        y_g = train[s]['Eg'].values
        y_f = train[s]['Ef'].values
        
        df_test[s] = data.groupby('sg').get_group(s)
        X_test = df_test[s].drop(test_ex_feature, axis=1).values
        
        params_g = out_rfr[s]["results_g"].groupby('rank_test_score').get_group(1)
        params_f = out_rfr[s]["results_f"].groupby('rank_test_score').get_group(1)
        
        print(s)        
        y_pred_g = np.array([]).reshape(0,X_test.shape[0])
        for i, param in enumerate(params_g["params"]):
            print("g", i)
            best_g = RFR(**param).fit(X, y_g)
            y_pred_g = np.vstack([y_pred_g, best_g.predict(X_test)])
        
        display(pd.DataFrame(y_pred_g))
        df_test[s]['std_g'] = np.std(y_pred_g, axis=0)
        df_test[s]['bandgap_energy_ev'] = np.mean(y_pred_g, axis=0)
        
        
        y_pred_f = np.array([]).reshape(0,X_test.shape[0])
        for j, param in enumerate(params_f["params"]):
            print("f", j)
            best_f = RFR(**param).fit(X, y_f)
            y_pred_f = np.vstack([y_pred_f, best_f.predict(X_test)])
            
        display(pd.DataFrame(y_pred_f))
        df_test[s]['std_f'] = np.std(y_pred_f, axis=0)
        df_test[s]['formation_energy_ev_natom'] = np.mean(y_pred_f, axis=0)
        
    df_pred = pd.concat([df_test[s] for s in sg]).sort_index()
    return df_pred, df_pred[['formation_energy_ev_natom', 'bandgap_energy_ev']]
        

pred = predict(data_test)
display(pred[1])


12
g 0
g 1
g 2
g 3
g 4
g 5
g 6
g 7
g 8
g 9
g 10
g 11
g 12


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,3.072782,1.606327,1.413347,1.668617,2.576652,0.890023,1.952213,1.819509,3.867137,1.853821,...,1.705619,1.598647,2.503754,1.428090,2.222576,1.832753,1.771315,3.250780,2.894303,1.340240
1,3.086714,1.607482,1.409773,1.659486,2.572435,0.877121,1.952838,1.819077,3.884434,1.844191,...,1.707461,1.593779,2.507935,1.424344,2.239730,1.825055,1.768680,3.239942,2.892770,1.347899
2,3.107239,1.609897,1.416846,1.664602,2.584871,0.883871,1.950726,1.825443,3.795853,1.844574,...,1.698906,1.606738,2.527640,1.422971,2.224411,1.838693,1.761994,3.233056,2.895209,1.320950
3,3.093344,1.610685,1.425051,1.671356,2.572611,0.885150,1.951807,1.829612,3.793344,1.854540,...,1.699275,1.609597,2.521658,1.415666,2.229675,1.816787,1.766357,3.232685,2.896695,1.320977
4,3.110010,1.602620,1.420482,1.662691,2.563998,0.870243,1.956730,1.821041,3.799186,1.858212,...,1.692240,1.606153,2.529813,1.418075,2.222777,1.831105,1.772123,3.237601,2.898802,1.326982
5,3.103220,1.608562,1.415826,1.666644,2.588659,0.882293,1.950720,1.827409,3.787835,1.839040,...,1.696543,1.605121,2.530095,1.426798,2.227149,1.828519,1.765311,3.233413,2.895554,1.325974
6,3.107120,1.607899,1.416256,1.666398,2.588699,0.880136,1.949342,1.829930,3.788078,1.847426,...,1.697694,1.604550,2.534813,1.427986,2.226049,1.844944,1.762944,3.232604,2.894188,1.322944
7,3.102886,1.608061,1.412876,1.667347,2.591670,0.879883,1.950900,1.826902,3.788459,1.848876,...,1.698497,1.605516,2.535095,1.417700,2.227864,1.850998,1.763647,3.232265,2.896004,1.323153
8,3.079298,1.612098,1.406510,1.660479,2.588354,0.879435,1.954755,1.831580,3.858040,1.848532,...,1.713789,1.591717,2.518284,1.435618,2.237681,1.853196,1.762457,3.242436,2.894311,1.345387
9,3.113018,1.616501,1.408985,1.675040,2.593853,0.880261,1.947785,1.824924,3.820807,1.850740,...,1.692988,1.605143,2.540445,1.411419,2.225867,1.868977,1.759401,3.226584,2.889254,1.314256


/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


f 0
f 1
f 2
f 3
f 4
f 5
f 6
f 7
f 8
f 9
f 10
f 11
f 12


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.028026,0.139015,0.198013,0.185100,0.033627,0.198647,0.155705,0.174590,0.031311,0.174369,...,0.034443,0.144797,0.018028,0.157285,0.098410,0.306201,0.124904,0.039501,0.033525,0.028596
1,0.031129,0.133689,0.188676,0.178055,0.034452,0.215030,0.153145,0.171983,0.027432,0.176526,...,0.041870,0.148436,0.018307,0.168839,0.096302,0.292514,0.121475,0.039579,0.033554,0.022983
2,0.030618,0.136675,0.189412,0.179839,0.031852,0.216224,0.150507,0.169774,0.024181,0.176120,...,0.043333,0.144958,0.015592,0.167628,0.089414,0.298334,0.123199,0.038545,0.032625,0.023445
3,0.030097,0.137152,0.193893,0.178723,0.033231,0.211541,0.151647,0.173269,0.028147,0.174538,...,0.041787,0.146526,0.018553,0.158742,0.092880,0.296003,0.125091,0.038788,0.033623,0.023936
4,0.029676,0.136309,0.195199,0.179748,0.033486,0.212908,0.151573,0.173943,0.027443,0.175070,...,0.041619,0.147749,0.018977,0.156908,0.092575,0.296842,0.125044,0.038722,0.033834,0.023999
5,0.029832,0.136516,0.194625,0.179415,0.033650,0.212551,0.151579,0.173070,0.028213,0.174652,...,0.041334,0.147091,0.019517,0.154985,0.094527,0.296437,0.124341,0.038862,0.033762,0.023991
6,0.029123,0.139140,0.195408,0.182726,0.033289,0.206132,0.153125,0.173259,0.028652,0.172427,...,0.033533,0.144113,0.017191,0.158480,0.100937,0.303370,0.123897,0.039310,0.033016,0.027962
7,0.029876,0.136495,0.194255,0.179590,0.033547,0.213119,0.151917,0.173565,0.027881,0.174882,...,0.041279,0.146708,0.019098,0.156061,0.092490,0.295550,0.124881,0.038781,0.033687,0.024101
8,0.026325,0.140148,0.201620,0.188783,0.033266,0.187483,0.155278,0.174961,0.027291,0.174451,...,0.021924,0.142335,0.012077,0.159954,0.114124,0.312385,0.124750,0.039508,0.033117,0.030578
9,0.029400,0.140045,0.191494,0.182289,0.034750,0.217185,0.152429,0.173997,0.025849,0.169423,...,0.042877,0.146076,0.019236,0.162348,0.093961,0.298854,0.126180,0.038357,0.033682,0.024342


...

/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


f 0
f 1
f 2
f 3
f 4
f 5
f 6
f 7
f 8
f 9
f 10
f 11
f 12


,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
0,0.189879,0.299171,0.174623,0.318070,0.169937,0.322205,0.332696,0.349802,0.244577,0.317994,...,0.350447,0.252013,0.327661,0.325439,0.336414,0.204495,0.320289,0.329610,0.182546,0.189717
1,0.164108,0.308562,0.126586,0.287483,0.125901,0.257623,0.331815,0.369562,0.234377,0.324196,...,0.346055,0.298843,0.338834,0.352267,0.308061,0.247363,0.334469,0.356265,0.127134,0.151685
2,0.167280,0.302297,0.133652,0.284143,0.126631,0.269793,0.331890,0.366721,0.239131,0.323892,...,0.345710,0.287516,0.339011,0.352348,0.305556,0.246463,0.334116,0.355381,0.141493,0.160408
3,0.166888,0.302783,0.134547,0.284439,0.127661,0.266054,0.332857,0.368218,0.234829,0.324991,...,0.346767,0.291835,0.338866,0.354168,0.307748,0.250462,0.332814,0.354881,0.139573,0.159027
4,0.164506,0.300798,0.133734,0.287734,0.125969,0.271677,0.331299,0.353252,0.240404,0.324398,...,0.348185,0.290262,0.336047,0.358330,0.300513,0.244078,0.331418,0.357202,0.142764,0.160132
5,0.167975,0.300449,0.133433,0.284020,0.126512,0.270316,0.333236,0.365655,0.234319,0.325127,...,0.347441,0.289133,0.341601,0.351948,0.308130,0.246450,0.333919,0.354253,0.143774,0.160967
6,0.172124,0.304252,0.129712,0.284143,0.122995,0.273194,0.335445,0.347627,0.233238,0.319832,...,0.345835,0.286556,0.345370,0.349616,0.312489,0.245767,0.337442,0.348339,0.147074,0.161437
7,0.168373,0.299190,0.133607,0.285037,0.127408,0.270202,0.332592,0.365213,0.231516,0.324501,...,0.346399,0.289635,0.341820,0.352742,0.308786,0.248562,0.334048,0.355062,0.143848,0.161586
8,0.166545,0.299141,0.132215,0.286824,0.127566,0.270192,0.332770,0.364951,0.233082,0.324861,...,0.347494,0.289087,0.341576,0.353493,0.309268,0.248626,0.333257,0.353544,0.141630,0.160917
9,0.163264,0.303731,0.141439,0.281993,0.128757,0.258959,0.333496,0.370501,0.226047,0.323888,...,0.339745,0.292678,0.334611,0.360415,0.307230,0.262078,0.336234,0.352899,0.145479,0.167458


/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,formation_energy_ev_natom,bandgap_energy_ev
id,,
1,0.197884,1.611268
2,0.071999,3.734608
3,0.152509,3.514554
4,0.029596,3.097736
5,0.137286,1.608834
6,0.044976,4.547722
7,0.070602,3.239355
8,0.133859,2.077048
9,0.067230,2.700825


In [211]:
display(pred[0])

,sg,n_atoms,x_Al,x_Ga,x_In,a,b,c,alpha,beta,...,avg_IP,avg_LUMO,avg_mass,avg_rd_max,avg_rp_max,avg_rs_max,std_g,bandgap_energy_ev,std_f,formation_energy_ev_natom
id,,,,,,,,,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,...,-0.499489,0.032240,6.748878,0.197757,0.120840,0.091832,0.004168,1.611268,0.002318,0.197884
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,...,-0.603143,0.032139,3.924055,0.299251,0.151839,0.114075,0.005185,3.734608,0.001901,0.071999
3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,...,-0.595236,0.039659,5.049802,0.285567,0.150554,0.113952,0.010559,3.514554,0.005024,0.152509
4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,...,-0.566828,0.025803,4.466880,0.263468,0.139404,0.104507,0.014237,3.097736,0.001260,0.029596
5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,...,-0.495524,0.030406,6.562892,0.196483,0.119535,0.090693,0.003092,1.608834,0.001752,0.137286
6,33,40.0,1.0000,0.0000,0.0000,4.8885,8.4018,9.0344,90.0061,90.0024,...,-0.623075,0.039670,2.908569,0.335253,0.161698,0.121812,0.060617,4.547722,0.000383,0.044976
7,167,30.0,0.3333,0.6667,0.0000,4.9772,4.9768,13.4593,89.9952,90.0061,...,-0.603215,0.021750,5.764347,0.257334,0.144081,0.107714,0.001055,3.239355,0.001375,0.070602
8,33,80.0,0.1875,0.6875,0.1250,10.2955,8.8306,9.4505,89.9988,90.0014,...,-0.537799,0.022835,6.270590,0.215143,0.127503,0.095787,0.005732,2.077048,0.001668,0.133859
9,33,80.0,0.3125,0.6875,0.0000,10.0984,8.6759,9.2968,89.9995,90.0021,...,-0.570277,0.020073,5.536163,0.241308,0.135847,0.101532,0.007913,2.700825,0.001447,0.067230


In [212]:
pred[1].to_csv('submission_rf_sg.csv')

In [84]:
train = {}
out_gbr = {}
avg_rmsle_gbr = []
for s in sg:
    train[s] = df.groupby('sg').get_group(s)
    out_gbr[s] = select_model(train[s], 'GBR')
    avg_rmsle_gbr = np.append(avg_rmsle_gbr, out_gbr[s]["avg_rmsle"]) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] learning_rate=0.225863508425, max_depth=29, min_weight_fraction_leaf=0.0256649720841, max_features=3, min_samples_split=13, min_samples_leaf=23, n_estimators=1269 
[CV] learning_rate=0.225863508425, max_depth=29, min_weight_fraction_leaf=0.0256649720841, max_features=3, min_samples_split=13, min_samples_leaf=23, n_estimators=1269 
[CV] learning_rate=0.225863508425, max_depth=29, min_weight_fraction_leaf=0.0256649720841, max_features=3, min_samples_split=13, min_samples_leaf=23, n_estimators=1269 
[CV] learning_rate=0.225863508425, max_depth=29, min_weight_fraction_leaf=0.0256649720841, max_features=3, min_samples_split=13, min_samples_leaf=23, n_estimators=1269 
[CV]  learning_rate=0.225863508425, max_depth=29, min_weight_fraction_leaf=0.0256649720841, max_features=3, min_samples_split=13, min_samples_leaf=23, n_estimators=1269, score=-0.05705970386821709, total=   0.7s
[CV]  learning_rate=0.225863508425, max_depth=29, mi

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    1.3s


[CV]  learning_rate=0.325783764673, max_depth=34, min_weight_fraction_leaf=0.269701791048, max_features=9, min_samples_split=12, min_samples_leaf=11, n_estimators=3267, score=-0.07652680312816687, total=   1.3s
[CV] learning_rate=0.325783764673, max_depth=34, min_weight_fraction_leaf=0.269701791048, max_features=9, min_samples_split=12, min_samples_leaf=11, n_estimators=3267 
[CV]  learning_rate=0.325783764673, max_depth=34, min_weight_fraction_leaf=0.269701791048, max_features=9, min_samples_split=12, min_samples_leaf=11, n_estimators=3267, score=-0.06543970572474528, total=   1.3s
[CV] learning_rate=0.385480067389, max_depth=28, min_weight_fraction_leaf=0.0974529256226, max_features=10, min_samples_split=7, min_samples_leaf=40, n_estimators=2207 
[CV]  learning_rate=0.325783764673, max_depth=34, min_weight_fraction_leaf=0.269701791048, max_features=9, min_samples_split=12, min_samples_leaf=11, n_estimators=3267, score=-0.06164017038611543, total=   1.3s
[CV] learning_rate=0.385480067

[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.2s


[CV] learning_rate=0.137935755857, max_depth=34, min_weight_fraction_leaf=0.386468100589, max_features=13, min_samples_split=7, min_samples_leaf=29, n_estimators=2157 
[CV]  learning_rate=0.385480067389, max_depth=28, min_weight_fraction_leaf=0.0974529256226, max_features=10, min_samples_split=7, min_samples_leaf=40, n_estimators=2207, score=-0.06878350544768248, total=   1.2s
[CV] learning_rate=0.137935755857, max_depth=34, min_weight_fraction_leaf=0.386468100589, max_features=13, min_samples_split=7, min_samples_leaf=29, n_estimators=2157 
[CV]  learning_rate=0.137935755857, max_depth=34, min_weight_fraction_leaf=0.386468100589, max_features=13, min_samples_split=7, min_samples_leaf=29, n_estimators=2157, score=-0.0933199711635737, total=   0.8s
[CV] learning_rate=0.137935755857, max_depth=34, min_weight_fraction_leaf=0.386468100589, max_features=13, min_samples_split=7, min_samples_leaf=29, n_estimators=2157 
[CV]  learning_rate=0.385480067389, max_depth=28, min_weight_fraction_leaf

...

[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   11.1s finished


Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] learning_rate=0.0564216060483, max_depth=39, min_weight_fraction_leaf=0.384811950279, max_features=2, min_samples_split=14, min_samples_leaf=46, n_estimators=1015 
[CV] learning_rate=0.0564216060483, max_depth=39, min_weight_fraction_leaf=0.384811950279, max_features=2, min_samples_split=14, min_samples_leaf=46, n_estimators=1015 
[CV] learning_rate=0.0564216060483, max_depth=39, min_weight_fraction_leaf=0.384811950279, max_features=2, min_samples_split=14, min_samples_leaf=46, n_estimators=1015 
[CV] learning_rate=0.0564216060483, max_depth=39, min_weight_fraction_leaf=0.384811950279, max_features=2, min_samples_split=14, min_samples_leaf=46, n_estimators=1015 
[CV]  learning_rate=0.0564216060483, max_depth=39, min_weight_fraction_leaf=0.384811950279, max_features=2, min_samples_split=14, min_samples_leaf=46, n_estimators=1015, score=-0.0529170070258947, total=   0.3s
[CV] learning_rate=0.0564216060483, max_depth=39, min

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.6s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:    0.7s remaining:    0.3s


[CV]  learning_rate=0.122359395053, max_depth=13, min_weight_fraction_leaf=0.00296071138043, max_features=2, min_samples_split=15, min_samples_leaf=49, n_estimators=1177, score=-0.0530816474886018, total=   0.3s
[CV]  learning_rate=0.122359395053, max_depth=13, min_weight_fraction_leaf=0.00296071138043, max_features=2, min_samples_split=15, min_samples_leaf=49, n_estimators=1177, score=-0.055351464806949705, total=   0.3s


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.0s finished


In [85]:
print(avg_rmsle_gbr, avg_rmsle_gbr.mean())

[ 0.04600574  0.0267319   0.02391975  0.07989842  0.02685562  0.11199274] 0.0525673631241


In [109]:
for s in sg:
    display(out_gbr[s]["results_g"].head())
    print(s, out_gbr[s]["best_g"])

,mean_fit_time,mean_score_time,mean_test_score,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_min_weight_fraction_leaf,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
11,0.281737,0.002276,-0.062999,0.500000,2,2,50,15,0.000000,1000,"{'learning_rate': 0.5, 'max_depth': 2, 'min_we...",1,-0.056968,-0.066474,-0.061946,-0.076935,-0.052724,0.019042,0.000334,0.008332
19,1.542786,0.005545,-0.058801,0.018212,2,12,4,10,0.070126,3434,"{'learning_rate': 0.0182123032161, 'max_depth'...",1,-0.054264,-0.067095,-0.056706,-0.070137,-0.045779,0.159923,0.003192,0.008830
21,0.641022,0.002053,-0.055999,0.010000,2,15,2,15,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.054200,-0.066567,-0.051415,-0.065428,-0.042326,0.008395,0.000279,0.009058
25,2.345046,0.008567,-0.059507,0.010000,2,15,2,15,0.000000,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.053963,-0.067611,-0.056794,-0.073348,-0.045822,0.065824,0.002680,0.009798
28,0.565438,0.004157,-0.056286,0.010000,2,15,2,15,0.036336,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.053531,-0.066768,-0.052746,-0.065887,-0.042441,0.017143,0.003516,0.009080


12 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=2,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=2,
             min_samples_split=15, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


,mean_fit_time,mean_score_time,mean_test_score,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_min_weight_fraction_leaf,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
11,0.619093,0.001597,-0.031562,0.01,2,15,2,15,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.027085,-0.034322,-0.035572,-0.031645,-0.029204,0.062638,0.000376,0.003143
15,0.317244,0.002085,-0.031980,0.01,2,2,2,2,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.029110,-0.032957,-0.036254,-0.030949,-0.030651,0.033778,0.000351,0.002460
20,0.829303,0.002462,-0.031674,0.01,2,15,2,15,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.027244,-0.034618,-0.035670,-0.031583,-0.029273,0.024691,0.000256,0.003166
26,2.451045,0.006459,-0.032419,0.01,2,15,2,2,0.000000,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.028088,-0.036958,-0.037813,-0.030734,-0.028499,0.026505,0.000151,0.004164
32,2.494375,0.008800,-0.032288,0.01,2,15,2,15,0.060988,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.028552,-0.035845,-0.035450,-0.030920,-0.030676,0.044793,0.003075,0.002870


33 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=15,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=2,
             min_samples_split=15, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


,mean_fit_time,mean_score_time,mean_test_score,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_min_weight_fraction_leaf,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
14,1.457326,0.004836,-0.029386,0.01,2,15,2,15,0.121655,3034,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.029830,-0.030994,-0.027331,-0.030081,-0.028685,0.012833,0.000363,0.001264
17,0.496288,0.002384,-0.029616,0.50,2,15,2,2,0.136210,1000,"{'learning_rate': 0.5, 'max_depth': 2, 'min_we...",1,-0.030906,-0.032407,-0.028451,-0.029431,-0.026849,0.014926,0.000398,0.001921
29,0.670541,0.003794,-0.026596,0.01,2,2,2,15,0.047814,2210,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.029851,-0.028084,-0.024830,-0.026099,-0.024084,0.008285,0.000214,0.002120
49,1.654802,0.005626,-0.029569,0.50,2,15,2,8,0.035930,4000,"{'learning_rate': 0.5, 'max_depth': 2, 'min_we...",1,-0.033571,-0.032571,-0.026282,-0.028837,-0.026545,0.291590,0.001742,0.003017
37,1.271646,0.012500,-0.030325,0.01,6,2,2,15,0.139145,3701,"{'learning_rate': 0.01, 'max_depth': 6, 'min_w...",1,-0.032814,-0.032771,-0.028318,-0.027965,-0.029749,0.013026,0.001833,0.002106


...

194 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=2,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=2,
             min_samples_split=15,
             min_weight_fraction_leaf=0.066954605356194491,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


,mean_fit_time,mean_score_time,mean_test_score,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_min_weight_fraction_leaf,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
21,0.655597,0.002981,-0.036515,0.011143,2,13,31,13,0.001144,1164,"{'learning_rate': 0.0111427276492, 'max_depth'...",1,-0.055303,-0.028474,-0.043254,-0.027532,-0.028011,0.018242,0.001030,0.011100
36,2.405969,0.008453,-0.036316,0.010000,2,15,2,15,0.000000,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.053295,-0.029873,-0.044224,-0.028540,-0.025649,0.052427,0.001848,0.010645
45,1.341887,0.012772,-0.035911,0.010000,2,2,2,15,0.000000,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.053058,-0.028914,-0.044047,-0.028160,-0.025374,0.012137,0.006798,0.010773
49,2.071603,0.007467,-0.036392,0.010000,2,15,2,2,0.000000,4000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.053493,-0.029748,-0.044441,-0.028602,-0.025676,0.422393,0.002780,0.010741
17,2.659369,0.013042,-0.037382,0.018511,4,13,2,6,0.000538,2943,"{'learning_rate': 0.0185108763189, 'max_depth'...",1,-0.053920,-0.031390,-0.045129,-0.030093,-0.026376,0.061331,0.005904,0.010428


206 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=2,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=2,
             min_samples_split=15, min_weight_fraction_leaf=0.0,
             n_estimators=4000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


,mean_fit_time,mean_score_time,mean_test_score,param_learning_rate,param_max_depth,param_max_features,param_min_samples_leaf,param_min_samples_split,param_min_weight_fraction_leaf,param_n_estimators,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,std_fit_time,std_score_time,std_test_score
13,0.440353,0.002308,-0.177866,0.01,2,7,50,2,0.096531,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.189203,-0.185003,-0.174403,-0.166251,-0.174280,0.015976,0.000682,0.008235
17,0.369286,0.002265,-0.177030,0.01,2,2,50,15,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.189826,-0.185091,-0.171647,-0.166290,-0.172097,0.023394,0.000195,0.008917
26,0.378035,0.004699,-0.176572,0.01,2,2,50,2,0.169791,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.189827,-0.183785,-0.172020,-0.166744,-0.170278,0.017057,0.004083,0.008773
29,0.529830,0.002024,-0.178323,0.01,2,15,50,15,0.235789,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.193066,-0.184449,-0.173565,-0.168768,-0.171558,0.021689,0.000593,0.009110
35,0.296704,0.001971,-0.177257,0.01,2,2,50,2,0.000000,1000,"{'learning_rate': 0.01, 'max_depth': 2, 'min_w...",1,-0.189039,-0.184332,-0.171352,-0.167221,-0.174176,0.018812,0.000314,0.008176


227 GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=2,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=50,
             min_samples_split=2,
             min_weight_fraction_leaf=0.16979113976006172,
             n_estimators=1000, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)


-0.07414774129996389

In [213]:
data_test = pd.read_csv('test_new_features.csv', index_col='id')

def predict(data):    
    best_model_g = {}
    best_model_f = {}    
    train_ex_feature = ['sg', 'Eg', 'Ef', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al',]
    test_ex_feature = ['sg', 'alpha_r', 'beta_r', 'gamma_r', 'x_In', 'x_Al']
    df_test = {}
    
    for s in sg:
        X = train[s].drop(train_ex_feature, axis=1).values
        y_g = train[s]['Eg'].values
        y_f = train[s]['Ef'].values
        
        df_test[s] = data.groupby('sg').get_group(s)
        X_test = df_test[s].drop(test_ex_feature, axis=1).values
        
        params_g = out_gbr[s]["results_g"].groupby('rank_test_score').get_group(1)
        params_f = out_gbr[s]["results_f"].groupby('rank_test_score').get_group(1)
        
        print(s)        
        y_pred_g = np.array([]).reshape(0,X_test.shape[0])
        for i, param in enumerate(params_g["params"]):
            print("g", i)
            best_g = GBR(**param).fit(X, y_g)
            y_pred_g = np.vstack([y_pred_g, best_g.predict(X_test)])
        
        display(pd.DataFrame(y_pred_g))
        df_test[s]['std_g'] = np.std(y_pred_g, axis=0)
        df_test[s]['bandgap_energy_ev'] = np.mean(y_pred_g, axis=0)
             
        y_pred_f = np.array([]).reshape(0,X_test.shape[0])
        for j, param in enumerate(params_f["params"]):
            print("f", j)
            best_f = GBR(**param).fit(X, y_f)
            y_pred_f = np.vstack([y_pred_f, best_f.predict(X_test)])
            
        display(pd.DataFrame(y_pred_f))
        df_test[s]['std_f'] = np.std(y_pred_f, axis=0)
        df_test[s]['formation_energy_ev_natom'] = np.mean(y_pred_f, axis=0)
        
    df_pred = pd.concat([df_test[s] for s in sg]).sort_index()
    return df_pred, df_pred[['formation_energy_ev_natom', 'bandgap_energy_ev']]
        

pred = predict(data_test)
display(pred[1])



12
g 0
g 1
g 2
g 3
g 4
g 5
g 6
g 7
g 8
g 9
g 10
g 11
g 12


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,3.068073,1.609244,1.390775,1.664300,2.251621,0.974819,1.933995,1.784799,3.927756,2.013525,...,1.715657,1.758216,2.366127,1.315480,2.277062,2.365589,1.764274,3.256771,2.897380,1.345027
1,3.087953,1.599849,1.414097,1.701166,2.551589,0.935136,1.928228,1.825622,3.965206,1.904091,...,1.713139,1.653905,2.546678,1.332415,2.251741,1.756816,1.756856,3.280287,2.892352,1.345995
2,3.064458,1.584342,1.403258,1.693604,2.549414,0.888829,1.950125,1.814482,3.938994,1.898510,...,1.688800,1.629631,2.544529,1.369753,2.205205,1.844382,1.752700,3.246444,2.906358,1.331031
3,3.086147,1.592892,1.404132,1.691917,2.559673,0.892582,1.942795,1.813517,3.981089,1.940724,...,1.721763,1.653318,2.548364,1.373870,2.237327,1.920303,1.758461,3.250926,2.902772,1.343192
4,3.062343,1.594703,1.414833,1.709670,2.514526,0.914802,1.946568,1.810487,3.850160,1.877724,...,1.695824,1.629563,2.507563,1.359485,2.216197,1.806326,1.754488,3.238307,2.909909,1.330224
5,3.047417,1.594194,1.416771,1.713355,2.540473,0.940447,1.939959,1.823361,3.776293,1.926625,...,1.698477,1.629190,2.530516,1.367041,2.221249,1.866922,1.751046,3.316043,2.898087,1.337686
6,3.052072,1.602039,1.410761,1.705628,2.486613,0.888969,1.940339,1.793064,3.936548,1.899902,...,1.702772,1.627690,2.483859,1.392317,2.237838,1.851250,1.757280,3.235727,2.899168,1.332770
7,3.062152,1.602018,1.405236,1.701872,2.483638,0.880387,1.944454,1.799977,3.950242,1.900818,...,1.695485,1.634827,2.472917,1.389516,2.258212,1.866992,1.756475,3.237932,2.897681,1.336274
8,3.052271,1.604341,1.404859,1.711542,2.503796,0.878450,1.953959,1.809134,3.923356,1.900283,...,1.696255,1.636098,2.503796,1.405641,2.234726,1.821847,1.753480,3.239278,2.901971,1.331752
9,3.146618,1.588249,1.413208,1.697921,2.466942,0.930415,1.951918,1.810171,3.677989,1.903639,...,1.701849,1.614581,2.466827,1.322536,2.249209,1.842723,1.742482,3.336016,2.904890,1.341428


/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/qiuruihao/miniconda2/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


f 0
f 1
f 2
f 3
f 4
f 5
f 6
f 7
f 8
f 9
f 10
f 11
f 12


,0,1,2,3,4,5,6,7,8,9,...,87,88,89,90,91,92,93,94,95,96
0,0.027524,0.132058,0.195055,0.160050,0.032223,0.173666,0.149784,0.182024,0.032720,0.182735,...,0.022020,0.132287,0.021692,0.142593,0.104838,0.283157,0.130105,0.035344,0.034494,0.036059
1,0.027477,0.132542,0.195957,0.159232,0.032629,0.171142,0.150556,0.182986,0.032522,0.183576,...,0.022137,0.131149,0.021378,0.142659,0.103723,0.281592,0.130092,0.035940,0.034072,0.035894
2,0.028124,0.136626,0.196782,0.163257,0.029859,0.181366,0.151229,0.187425,0.054261,0.190643,...,0.017740,0.134643,0.029859,0.146410,0.103772,0.237909,0.130710,0.032206,0.031531,0.047555
3,0.030616,0.140202,0.194383,0.170100,0.037794,0.163611,0.152403,0.178838,0.028004,0.174608,...,0.024058,0.140866,0.021793,0.118890,0.108392,0.250839,0.124347,0.037825,0.033998,0.034783
4,0.025494,0.146526,0.204711,0.161290,0.029214,0.171319,0.157652,0.195188,0.035180,0.200979,...,0.037209,0.123923,0.028896,0.117362,0.095760,0.200656,0.124175,0.037190,0.034559,0.051725
5,0.026521,0.142658,0.202003,0.197443,0.019288,0.177382,0.162144,0.182169,0.030285,0.190079,...,0.023139,0.133318,0.007814,0.163181,0.121259,0.241176,0.123564,0.038425,0.033213,0.034872
6,0.024523,0.150931,0.206149,0.166572,0.033417,0.182628,0.160011,0.192208,0.032908,0.191982,...,0.031459,0.124615,0.032147,0.114120,0.106633,0.185657,0.120575,0.035650,0.034834,0.046668
7,0.024139,0.137755,0.206830,0.158250,0.017933,0.157870,0.151746,0.192638,0.055220,0.213237,...,0.046948,0.135978,0.016361,0.143766,0.091645,0.213807,0.126493,0.035180,0.034662,0.062996
8,0.024600,0.141750,0.201425,0.202600,0.017011,0.177294,0.158300,0.180782,0.026987,0.159230,...,0.019500,0.152570,0.010988,0.095400,0.133700,0.287906,0.124420,0.039400,0.033171,0.032400
9,0.025071,0.141740,0.201365,0.202530,0.028303,0.113027,0.158184,0.170833,0.027416,0.177761,...,0.019538,0.115837,0.014244,0.117654,0.133686,0.188541,0.124409,0.039393,0.033179,0.032378


...

227
g 0
g 1
g 2
g 3
g 4
g 5
g 6
g 7
g 8
g 9
g 10
g 11
g 12


,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
0,1.688260,0.582294,2.777139,0.501120,2.926019,0.518675,1.045967,1.261359,1.519296,0.758030,...,1.333803,1.524349,0.923716,0.375865,1.231457,2.423884,0.458097,0.503959,1.908271,1.637735
1,1.706229,0.612009,2.767018,0.540591,2.896836,0.524181,1.067388,1.295926,1.506423,0.777631,...,1.353753,1.591241,0.867831,0.385584,1.320417,2.417857,0.450978,0.553410,1.964606,1.663319
2,1.696974,0.528050,2.863338,0.594283,2.840557,0.584401,1.061139,1.290933,1.503856,0.795176,...,1.349296,1.604925,0.884368,0.415082,1.328772,2.396759,0.487000,0.577608,1.893161,1.704395
3,1.881865,0.497215,2.705663,0.616495,2.705918,0.614612,1.130872,1.225489,1.459629,0.749291,...,1.365471,1.546758,0.890052,0.415557,1.293849,2.488414,0.511201,0.532320,2.044478,1.529157
4,1.672053,0.630547,2.766238,0.536804,2.914675,0.516916,1.068163,1.304633,1.508530,0.764283,...,1.339458,1.567748,0.870972,0.374303,1.311661,2.413204,0.455706,0.536833,1.930260,1.651691
5,1.787501,0.538504,2.751347,0.615909,2.746200,0.611466,1.084756,1.305988,1.463120,0.742132,...,1.399726,1.596186,0.829419,0.436973,1.340735,2.426404,0.476341,0.577141,1.920234,1.651585
6,1.694372,0.606993,2.776559,0.543258,2.892398,0.524654,1.058990,1.286895,1.506974,0.767252,...,1.343118,1.588972,0.867836,0.394724,1.321878,2.425161,0.459256,0.550700,1.957827,1.653272
7,1.741872,0.573675,2.783146,0.549733,2.753093,0.590847,1.083861,1.339552,1.464503,0.744073,...,1.391389,1.575730,0.949311,0.418898,1.424217,2.512912,0.512963,0.542754,2.014985,1.684176
8,1.970848,0.743294,2.471520,0.638170,2.694091,0.451193,1.051085,1.415084,1.915701,0.703179,...,1.523163,1.536683,0.743294,0.484392,1.356076,2.620132,0.484373,0.767975,1.851696,1.900119
9,1.831021,0.549907,2.667991,0.676713,2.689962,0.671500,1.128160,1.319473,1.454333,0.628859,...,1.419473,1.624444,0.816585,0.416198,1.432684,2.428159,0.481092,0.515471,1.950428,1.629387


f 0
f 1
f 2
f 3
f 4
f 5
f 6
f 7
f 8
f 9
f 10
f 11
f 12


,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
0,0.162813,0.315266,0.066970,0.294263,0.121067,0.266976,0.332051,0.377291,0.207996,0.321450,...,0.330655,0.370588,0.332271,0.367267,0.280510,0.210894,0.315500,0.344314,0.092195,0.160969
1,0.169802,0.293888,0.113524,0.291005,0.126142,0.250753,0.335473,0.376734,0.249837,0.319558,...,0.339172,0.312269,0.368746,0.347808,0.284298,0.267806,0.317605,0.346380,0.107768,0.145455
2,0.166632,0.301315,0.124979,0.292567,0.124497,0.250334,0.337048,0.370694,0.220340,0.323583,...,0.343037,0.307893,0.356383,0.340847,0.308548,0.271443,0.340794,0.357136,0.107644,0.140414
3,0.164424,0.296831,0.083179,0.290602,0.126751,0.275099,0.333816,0.366549,0.185314,0.316852,...,0.348658,0.326290,0.357587,0.368770,0.269217,0.232220,0.294795,0.354144,0.098554,0.140834
4,0.166266,0.309542,0.113358,0.294250,0.121067,0.253968,0.332050,0.379026,0.223734,0.321450,...,0.341255,0.325349,0.384019,0.314542,0.300361,0.273539,0.349591,0.346630,0.092200,0.154441
5,0.197083,0.316463,0.183521,0.301864,0.172119,0.312747,0.276119,0.318898,0.231084,0.276119,...,0.340927,0.256441,0.316463,0.355323,0.279587,0.264091,0.342209,0.353091,0.204349,0.154122
6,0.160211,0.328247,0.113221,0.294250,0.121067,0.236704,0.332050,0.374318,0.250826,0.321450,...,0.342649,0.309319,0.318805,0.366714,0.328306,0.252748,0.361564,0.358511,0.092200,0.143470
7,0.177458,0.302688,0.126722,0.285938,0.132047,0.287583,0.342807,0.349088,0.216487,0.317881,...,0.344078,0.306958,0.351228,0.335764,0.256738,0.232839,0.313875,0.351572,0.150490,0.151450
8,0.179239,0.305432,0.127645,0.285857,0.131377,0.287252,0.342611,0.348534,0.213857,0.317509,...,0.341174,0.303734,0.351835,0.338076,0.262316,0.231795,0.312326,0.352423,0.151913,0.148989
9,0.181986,0.301667,0.128975,0.286897,0.134237,0.287321,0.341977,0.341222,0.219655,0.317706,...,0.343708,0.302551,0.352942,0.337364,0.257784,0.231416,0.314070,0.351911,0.147584,0.149961


,formation_energy_ev_natom,bandgap_energy_ev
id,,
1,0.199167,1.598719
2,0.063314,3.763508
3,0.146422,3.522767
4,0.026427,3.072696
5,0.141000,1.598067
6,0.050925,4.370893
7,0.070100,3.240104
8,0.114242,2.060820
9,0.066772,2.702983


In [214]:
pred[1].to_csv('submission_gbr_sg.csv')

In [128]:
val = np.mean(val_g.reshape(6,10), axis=1)
print(val)
avg = []
w = []
for i, s in enumerate(sg):
    print(i, s)
    print(train[s].shape)
    print(val[i])
    w = np.append(w, train[s].shape[0])
print(w)
print(np.average(val, weights=w))

[-0.05310239 -0.03162358 -0.02584097 -0.10910967 -0.03421016 -0.17679476]
0 12
(358, 29)
-0.0531023891857
1 33
(432, 29)
-0.0316235776231
2 167
(374, 29)
-0.0258409669801
3 194
(353, 29)
-0.109109665085
4 206
(490, 29)
-0.0342101573863
5 227
(393, 29)
-0.176794760178
[ 358.  432.  374.  353.  490.  393.]
-0.0696231633986


In [105]:
np.mean(val_g.reshape(6,10), axis=1).mean()

-0.071856254015763885

In [ ]:
## XGB

from xgboost import XGBRegressor as XGBR

